In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/Hackathon/')

In [ ]:
import pandas as pd
import numpy as np

# Text cleaning

In [ ]:
import re

def clean_text(text):
    # Convert text to lowercase
    text = str(text).lower().strip()

    # Remove the \n at the end of each line in the file
    text = text.rstrip('\n')

    # Remove HTML tags and non-alphanumeric characters
    text = re.sub(r"<[^>]+>", "", text)
    text = re.sub(r"[.,;!?':()\[\]{}-]", " ", text)  # Keep selected punctuation marks, symbols and apostrophes

    # Remove excessive whitespace (more than one space)
    #text = re.sub(r"\s+", " ", text)

    text = text.encode("utf-8", errors="ignore").decode("utf-8")  # Corrected encoding

    return text

In [ ]:
from nltk.tokenize import word_tokenize

In [ ]:
DATA_PATH = '/content/drive/MyDrive/Colab Notebooks/Hackathon/'
df = pd.read_csv(DATA_PATH + 'data.csv')

In [ ]:
df.head()

,Unnamed: 0,target,source
0,0,Та пыгрисит маим вармаль э̄рнэ поратэт ат верм...,Те мальчики не выполнят задание в назначенный ...
1,1,"Ха̄йтыматэ тӯр ва̄тан ёхтыс, вит ва̄тан ха̄йтыс.","Бегая к берегу озера пришла, к воде подбежала."
2,2,Вит са̄мыл сунсым о̄нтыс,Вода прибывала на глазах
3,3,"Атаявев, акваг лылынг тагл ворн та тотавев.","Обнюхивает нас, живыми на кладбище уносит."
4,4,"Ман ты пӣлтал, веськат хумиюв нэтхуньт ат ёр...",Мы никогда не забудем этого честного человека.


In [ ]:
len(df)

81146

In [ ]:
df_cleaned = pd.concat([df['target'].apply(lambda text: clean_text(text)),
                        df['source'].apply(lambda text: clean_text(text))],
                        axis=1)

In [ ]:
df_cleaned.head()

,target,source
0,та пыгрисит маим вармаль э̄рнэ поратэт ат верм...,те мальчики не выполнят задание в назначенный ...
1,ха̄йтыматэ тӯр ва̄тан ёхтыс вит ва̄тан ха̄йтыс,бегая к берегу озера пришла к воде подбежала
2,вит са̄мыл сунсым о̄нтыс,вода прибывала на глазах
3,атаявев акваг лылынг тагл ворн та тотавев,обнюхивает нас живыми на кладбище уносит
4,ман ты пӣлтал веськат хумиюв нэтхуньт ат ёр...,мы никогда не забудем этого честного человека


# Tokenization

In [ ]:
from transformers import AutoTokenizer

In [ ]:
PRE_TRAINED_MODEL_NAME = "distilbert/distilbert-base-multilingual-cased"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [ ]:
tokenizer.vocab_size

119547

In [ ]:
tokenizer.all_special_tokens

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']

In [ ]:
print('[CLS] : ', tokenizer.cls_token_id)
print('[SEP] : ', tokenizer.sep_token_id)
print('[UNK] : ', tokenizer.unk_token_id)
print('[PAD] : ', tokenizer.pad_token_id)

[CLS] :  101
[SEP] :  102
[UNK] :  100
[PAD] :  0


In [ ]:
encoded_test = tokenizer.encode('Рӯпитанэ̄тэ сыс, хорамыӈ яныг кол ӯнттувес ос йильпи ищхӣпыӈ утыл ёвтвес.')

In [ ]:
tokenizer.decode(encoded_test)

'[CLS] Рӯпитанэ̄тэ сыс, [UNK] яныг кол ӯнттувес ос йильпи [UNK] утыл ёвтвес. [SEP]'

In [ ]:

from transformers import AutoModelForMaskedLM

pre_trained_model = AutoModelForMaskedLM.from_pretrained(PRE_TRAINED_MODEL_NAME)

embedding_layer = pre_trained_model.get_input_embeddings()

# These line just tells pytorch we don't intend to further train the embedding layer.
# So it freezes the layers knowledge, so we don't scatter it while our model is still starting to learn.
embedding_layer = embedding_layer.requires_grad_(False) # Мб нужно будет убрать, для лучшего результата

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

In [ ]:
print('Vocabulary Size :', tokenizer.vocab_size)
print('Embedding Layer :', embedding_layer)

Vocabulary Size : 119547
Embedding Layer : Embedding(119547, 768, padding_idx=0)


# Проверка работы токенайзера: в скольки предложениях есть токены, которые не известны токенайзеру

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_train, df_test_split = train_test_split(df, test_size=0.1, random_state=10)

In [ ]:
df_test, df_validation = train_test_split(df_test_split, test_size=0.5, random_state=10)

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
model_name = "facebook/nllb-200-distilled-600M"

tokenizer = AutoTokenizer.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
import re

def word_tokenize(text):
    """
    Split a text into words, numbers, and punctuation marks
    (for languages where words are separated by spaces)
    """
    return re.findall('(\w+|[^\w\s])', text)

smpl = df_train.sample(10000, random_state=1)
smpl['source_toks'] = smpl.source.apply(tokenizer.tokenize)
smpl['target_toks'] = smpl.target.apply(tokenizer.tokenize)
smpl['source_words'] = smpl.source.apply(word_tokenize)
smpl['target_words'] = smpl.target.apply(word_tokenize)

In [ ]:
smpl.head()

,Unnamed: 0,target,source,source_toks,target_toks,source_words,target_words
71754,19917,"Ос акв та̄рвитыӈ ва̄рмаль тот о̄лы, рӯпата щар...","Еще одно есть там трудное дело, совсем нет раб...","[▁Е, ще, ▁одно, ▁есть, ▁там, ▁трудно, е, ▁дело...","[▁Ос, ▁ак, в, ▁та, ̄, р, ви, ты, <unk>, ▁ва, ̄...","[Еще, одно, есть, там, трудное, дело, ,, совсе...","[Ос, акв, та, ̄, рвитыӈ, ва, ̄, рмаль, тот, о,..."
48454,766,"Хуль сэ̄ӈкв ке хо̄иглас, Хуль сэ̄ӈквы̄н хосгēгум.","Если возникали болезни, посланные Хулем, я сме...","[▁Если, ▁воз, ника, ли, ▁боле, зни, ,, ▁послан...","[▁Ху, ль, ▁с, э, ̄, <unk>, к, в, ▁ке, ▁хо, ̄, ...","[Если, возникали, болезни, ,, посланные, Хулем...","[Хуль, сэ, ̄, ӈкв, ке, хо, ̄, иглас, ,, Хуль, ..."
37549,37549,По̄рхыӈ ӯй по̄рханыл,"Их туловища, туловища животных","[▁Их, ▁ту, лови, ща, ,, ▁ту, лови, ща, ▁живот,...","[▁По, ̄, р, хы, <unk>, ▁ӯ, й, ▁по, ̄, р, ха, н...","[Их, туловища, ,, туловища, животных]","[По, ̄, рхыӈ, ӯй, по, ̄, рханыл]"
15321,15321,Ам ялпыӈ витуп нариӈ тӯрум котьлин.,На середину своего священного озера.,"[▁На, ▁се, реди, ну, ▁своего, ▁свя, щен, ного,...","[▁Ам, ▁ял, пы, <unk>, ▁ви, туп, ▁нари, <unk>, ...","[На, середину, своего, священного, озера, .]","[Ам, ялпыӈ, витуп, нариӈ, тӯрум, котьлин, .]"
43613,43613,Ты сēп тув пальтувын ю̄нтэ̄лын.,Этот карман туда к пальто пришей.,"[▁Этот, ▁кар, ман, ▁туда, ▁к, ▁паль, то, ▁при,...","[▁Ты, ▁с, ē, п, ▁ту, в, ▁паль, ту, в, ын, ▁ю, ...","[Этот, карман, туда, к, пальто, пришей, .]","[Ты, сēп, тув, пальтувын, ю, ̄, нтэ, ̄, лын, .]"


In [ ]:
stats = smpl[
    ['source_toks', 'target_toks', 'source_words', 'target_words']
].applymap(len).describe()
print(stats.source_toks['mean'] / stats.source_words['mean'])  # 2.0349
print(stats.target_toks['mean'] / stats.target_words['mean'])  # 2.4234
stats

1.6589278973197432
1.6734489121931118


<ipython-input-18-0f359d39c067>:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ].applymap(len).describe()


,source_toks,target_toks,source_words,target_words
count,10000.000000,10000.000000,10000.000000,10000.000000
mean,17.578000,28.021400,10.596000,16.744700
std,10.308964,16.748653,6.197372,10.523469
min,2.000000,3.000000,1.000000,1.000000
25%,10.000000,16.000000,6.000000,9.000000
50%,15.000000,24.000000,9.000000,14.000000
75%,22.000000,36.000000,14.000000,22.000000
max,135.000000,223.000000,76.000000,120.000000


In [ ]:
from tqdm.auto import tqdm, trange
import random
texts_with_unk = [
    text for text in tqdm(df.target)
    if tokenizer.unk_token_id in tokenizer(text).input_ids
]
print(len(texts_with_unk))
s = random.sample(texts_with_unk, 5)
print(s)


  0%|          | 0/81146 [00:00<?, ?it/s]

34058
['Наӈ исколан минуӈкве що̄питахтэ̄гын? Ам юн хультуӈкве номсэ̄гум.', 'Кур па̄лтыглаӈкве патэ̄гын, каша па̄йтуӈкве, ос а̄нум кашал тыттуӈкве патылын.', 'Нэ̄мхоты товлыӈ То̄рум.', '- Ти на̄ӈын чижыл а̄ти сё̄при ма̄тыл.', 'Наӈ вос ӯнттыслын']


Слишком много предложений в которых токенайзер не узнал слова и назначил UNK